# The Self-Correction Pattern - Building a Critique Loop

Hello everyone. Today, we'll cover a pattern that leverages the model's reasoning capabilities in a clever way to improve the quality of its own work. This is the **Self-Correction Pattern.**

The core idea is inspired by how humans improve their own work: through critique and refinement. We break the process into at least two distinct steps:

1. **Generation:** The model produces an initial draft of the solution.
2. **Critique & Refinement:** The model takes its own first draft as input, critiques it against a set of criteria, and then generates a superior, improved version.

## The Task and Helper Functions

First, let's set up our standard helper functions.

In [ ]:
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and code context are ready.")

## The First Draft (Generation)

First, we'll use a simple, single-shot prompt to ask for the function.

In [ ]:
prompt_v1 = [
    {
        "role": "user",
        "content": dedent("""
        Write a Python function named `validate_email` that takes and email string and
        returns True if valid, False otherwise. Output only the code.
        """)
    }
]

first_draft_code = get_completion(prompt_v1)

In [ ]:
print(first_draft_code)

assistant_response_v1 = [
    {
        "role": "assistant",
        "content": first_draft_code
    }
]

## The Second Draft (Critique & Refinement)

Now, we construct our second prompt. This prompt is more sophisticated. It includes the original request **and** the model's first draft. We assign it an "expert reviewer" persona and explicitly ask it to critique the draft and then provide an improved version.

In [ ]:
prompt_refinement = [
    {
        "role": "user",
        "content": dedent("""
        Given the information and previous exchanges, please perform the following two actions:

        1. Critique: Identify specific issues with the proposed solution, considering the solution domain.
        2. Refine: Provide an improved version that addresses these issues. Keep it more robust and
            production-ready.
        """)
    }
]

In [ ]:
final_conversation = prompt_v1 + assistant_response_v1 + prompt_refinement

revised_code = get_completion(final_conversation)

In [ ]:
display(Markdown(revised_code))